Tables and results describing h5 transform files

In [67]:
import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [11]:
bridge_list = ['JRC2018F_FAFB', 'JRC2018F_FCWB', 'JRC2018F_JFRC2010', 'JRC2018F_JFRC2013', 'JRC2018F_TEFOR']

In [37]:
sizes_data_f="/groups/saalfeld/home/bogovicj/dev/template/template-building-pub/scripts/h5Analysis/h5Sizes.csv"
sizes_h5 = pd.read_csv( sizes_data_f )
sizes_h5

,file,size(bytes)
0,JRC2018F_FAFB.h5,580398481
1,JRC2018F_FCWB.h5,1291412644
2,JRC2018F_JFRC2010.h5,1648355301
3,JRC2018F_JFRC2013.h5,1394903421
4,JRC2018F_TEFOR.h5,1623178929


Here I'll tabulate sizes of various transformations out of ants
These came from the transforms here:

`/groups/saalfeld/public/jrc2018/transformations`

Specifically, these subfolders / files:
* `JRC2018F_FAFB/*Warp.nii*`
* `jrc2018F-FCWB/*Warp.nii*`
* `jrc2018F-jfrc2010/*Warp.nii*`
* `jrc2018F-jfrc2013/*Warp.nii*`
* `JRC2018F_TEFOR/*Warp.nii*`


In [53]:
ants_transform_sizes = [
    ('JRC2018F_FAFB_Warp.nii',2833742131),
    ('JRC2018F_FAFB_InverseWarp.nii',2832463797),
    ('JRC2018F_FCWB_Warp.nii',7893413462),
    ('JRC2018F_FCWB_InverseWarp.nii',7897010532),
    ('JRC2018F_JFRC2010_Warp.nii',8133116764),
    ('JRC2018F_JFRC2010_InverseWarp.nii',8143715252),
    ('JRC2018F_JFRC2013_Warp.nii',8002888343),
    ('JRC2018F_JFRC2013_InverseWarp.nii',8003178281),
    ('JRC2018F_TEFOR_Warp.nii',8127594570),
    ('JRC2018F_TEFOR_InverseWarp.nii',8136532225),
]
ants_df_raw = pd.DataFrame( ants_transform_sizes )
ants_df_raw.columns = [ 'file', 'size(bytes)']

In [52]:
def bridge_index( x, bridge_list ):
    for i, b in enumerate( bridge_list ):
        if x.startswith( b ):
            return i

ants_df_raw['bridge_idx'] = ants_df_raw.apply( lambda x: (bridge_index(x['file'], bridge_list )), axis=1)

In [50]:
# Build a dataframe containing the combined size of the compressed inverse and forward fields
ants_df_data = []
for i in range( len(bridge_list)):
    ants_df_data += [ (bridge_list[i], ants_df_raw[ants_df_raw.bridge_idx == i ]['size(bytes)'].sum()) ] 


ants_df = pd.DataFrame( ants_df_data )
ants_df.columns = [ 'file', 'size(bytes)']

In [59]:
# Combine h5 size and ants dfield size tables
billion=1e9

sizes_h5['file'] = sizes_h5.apply( lambda x: x['file'].replace('.h5',''), axis=1 )
df = sizes_h5.set_index('file').join(ants_df.set_index('file'), rsuffix='_ants')

# Compute relative size
df['ratio'] = df.apply( lambda x: x['size(bytes)']/x['size(bytes)_ants'], axis=1)
df['h5 Size (GB)'] = df.apply( lambda x: x['size(bytes)']/billion, axis=1)
df['Size (GB)'] = df.apply( lambda x: x['size(bytes)_ants']/billion, axis=1)
df

,size(bytes),size(bytes)_ants,ratio,h5 Size (GB),Size (GB)
file,,,,,
JRC2018F_FAFB,580398481,5666205928,0.102432,0.580398,5.666206
JRC2018F_FCWB,1291412644,15790423994,0.081785,1.291413,15.790424
JRC2018F_JFRC2010,1648355301,16276832016,0.101270,1.648355,16.276832
JRC2018F_JFRC2013,1394903421,16006066624,0.087148,1.394903,16.006067
JRC2018F_TEFOR,1623178929,16264126795,0.099801,1.623179,16.264127


In [70]:
df.index.names = ['Transform']
df.columns= ['size(bytes)', 'size(bytes)_ants', 'Size Ratio', 'h5 Size (Gb)', 'Size (Gb)']

df_toWrite = df[[ 'Size (Gb)', 'h5 Size (Gb)','Size Ratio']]

h5size_table_f = 'h5SizeTable_%s.tex'%(datetime.date.today().strftime('%Y%m%d'))

# Should I write
if( True):
    print( 'writing : ', h5size_table_f )
    with open( h5size_table_f, 'w') as f:
        f.write( df_toWrite.to_latex())
        
df_toWrite

writing :  h5SizeTable_20190917.tex


,Size (Gb),h5 Size (Gb),Size Ratio
Transform,,,
JRC2018F_FAFB,5.666206,0.580398,0.102432
JRC2018F_FCWB,15.790424,1.291413,0.081785
JRC2018F_JFRC2010,16.276832,1.648355,0.101270
JRC2018F_JFRC2013,16.006067,1.394903,0.087148
JRC2018F_TEFOR,16.264127,1.623179,0.099801
